# Explore Gene Lists in Raw and Filtered Data

This notebook loads both raw and filtered data files to inspect and compare their gene lists.


In [1]:
import anndata as ad
import pandas as pd
import numpy as np

# Check anndata version
print(f"anndata version: {ad.__version__}")
print("Note: If you encounter IORegistryError, you may need to upgrade anndata:")
print("  pip install --upgrade anndata")


anndata version: 0.12.6
Note: If you encounter IORegistryError, you may need to upgrade anndata:
  pip install --upgrade anndata


/tmp/ipykernel_4176641/2052231776.py:6: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  print(f"anndata version: {ad.__version__}")


In [2]:
# !pip install --upgrade anndata

## ⚠️ Troubleshooting IORegistryError

If you get an `IORegistryError` when loading the filtered files, this means the files were created with a newer version of anndata than what's currently installed.

**Solution:** Update anndata by running this in your terminal:
```bash
pip install --upgrade anndata
```

Or in a notebook cell:
```python
!pip install --upgrade anndata
```

After upgrading, restart the kernel and re-run the cells.


## Define File Paths


## Upgrade anndata (if needed)

Uncomment and run the following cell if you encountered IORegistryError:


## Diagnostic: Test Loading Files Separately


In [3]:
# Test loading one raw and one filtered file separately to diagnose
print("Testing individual file loading...")
print("\n1. Testing RAW file (hepg2):")
try:
    test_raw = ad.read_h5ad('data/GSE264667_hepg2_raw_singlecell_01.h5ad')
    print(f"   ✓ SUCCESS: {test_raw.shape}")
except Exception as e:
    print(f"   ✗ FAILED: {type(e).__name__}")

print("\n2. Testing FILTERED file (hepg2):")
try:
    test_filtered = ad.read_h5ad('data/GSE264667_hepg2_raw_singlecell_01_filtered.h5ad')
    print(f"   ✓ SUCCESS: {test_filtered.shape}")
except Exception as e:
    print(f"   ✗ FAILED: {type(e).__name__}: {str(e)[:200]}")


Testing individual file loading...

1. Testing RAW file (hepg2):
   ✓ SUCCESS: (145473, 9624)

2. Testing FILTERED file (hepg2):
   ✓ SUCCESS: (96616, 9624)


## Understanding the Error

The `IORegistryError` with encoding type `'null'` version `'0.1.0'` means the filtered h5ad files contain a data encoding format that your current anndata version doesn't recognize.

**This typically happens when:**
- Files were created with anndata 0.12+ or a dev version
- Files were created with a different tool (like the `cell-load` CLI used in the filtering step)
- The `cell-load` package uses internal formats not compatible with standard anndata

**Solutions to try:**

1. **Install development version of anndata:**
   ```python
   !pip install --pre anndata
   ```

2. **Install anndata from GitHub (latest):**
   ```python
   !pip install git+https://github.com/scverse/anndata.git
   ```

3. **Check which Python environment created the files:**
   - The filtering was done with `uvx` which may have used a different Python/anndata version
   - You might need to use that same environment to read them

4. **Work with only RAW files for now:**
   - Skip the filtered files and work directly with raw data
   - You can do your own filtering with your current anndata version


In [ ]:
# Try installing latest anndata from GitHub (uncomment to run)
# !pip install --upgrade git+https://github.com/scverse/anndata.git
# After running, restart kernel and re-run from the top


In [3]:
# Uncomment the line below to upgrade anndata
# !pip install --upgrade anndata
# After running, restart kernel and re-run from the top


In [2]:
# Define raw and filtered file pairs
datasets = {
    'hepg2': {
        'raw': 'data/GSE264667_hepg2_raw_singlecell_01.h5ad',
        'filtered': 'data/GSE264667_hepg2_raw_singlecell_01_filtered.h5ad'
    },
    'jurkat': {
        'raw': 'data/GSE264667_jurkat_raw_singlecell_01.h5ad',
        'filtered': 'data/GSE264667_jurkat_raw_singlecell_01_filtered.h5ad'
    },
    'k562': {
        'raw': 'data/K562_essential_normalized_singlecell_01.h5ad',
        'filtered': 'data/K562_essential_normalized_singlecell_01_filtered.h5ad'
    },
    'rpe1': {
        'raw': 'data/rpe1_normalized_singlecell_01.h5ad',
        'filtered': 'data/rpe1_normalized_singlecell_01_filtered.h5ad'
    }
}


## Load All Datasets and Extract Basic Info


In [3]:
# Load all datasets with error handling
adata_dict = {}
failed_loads = []

for name, paths in datasets.items():
    print(f"Loading {name}...")
    try:
        adata_dict[name] = {
            'raw': ad.read_h5ad(paths['raw']),
            'filtered': ad.read_h5ad(paths['filtered'])
        }
        print(f"  Raw: {adata_dict[name]['raw'].shape}")
        print(f"  Filtered: {adata_dict[name]['filtered'].shape}")
    except Exception as e:
        print(f"  ERROR loading {name}: {type(e).__name__}: {str(e)[:100]}")
        failed_loads.append(name)
    print()

if failed_loads:
    print(f"\n⚠️  Warning: Failed to load {len(failed_loads)} dataset(s): {failed_loads}")
    print("\nThis is likely due to anndata version incompatibility.")
    print("Try running: pip install --upgrade anndata")
else:
    print("✓ All datasets loaded successfully!")


Loading hepg2...
  Raw: (145473, 9624)
  Filtered: (96616, 9624)

Loading jurkat...
  Raw: (262956, 8882)
  Filtered: (184470, 8882)

Loading k562...
  Raw: (310385, 8563)
  Filtered: (188590, 8563)

Loading rpe1...
  Raw: (247914, 8749)
  Filtered: (173737, 8749)

✓ All datasets loaded successfully!


## Compare Dataset Dimensions


In [4]:
# Create comparison table
if not adata_dict:
    print("⚠️  No datasets loaded. Please resolve loading errors first.")
else:
    comparison_data = []

    for name, adatas in adata_dict.items():
        raw = adatas['raw']
        filtered = adatas['filtered']
        
        comparison_data.append({
            'Dataset': name,
            'Raw Cells': raw.n_obs,
            'Filtered Cells': filtered.n_obs,
            'Cells Removed': raw.n_obs - filtered.n_obs,
            'Cells Retained (%)': f"{100 * filtered.n_obs / raw.n_obs:.1f}",
            'Raw Genes': raw.n_vars,
            'Filtered Genes': filtered.n_vars,
            'Genes Removed': raw.n_vars - filtered.n_vars,
            'Genes Retained (%)': f"{100 * filtered.n_vars / raw.n_vars:.1f}"
        })

    comparison_df = pd.DataFrame(comparison_data)
    print("\n=== Dataset Comparison ===")
    print(comparison_df.to_string(index=False))



=== Dataset Comparison ===
Dataset  Raw Cells  Filtered Cells  Cells Removed Cells Retained (%)  Raw Genes  Filtered Genes  Genes Removed Genes Retained (%)
  hepg2     145473           96616          48857               66.4       9624            9624              0              100.0
 jurkat     262956          184470          78486               70.2       8882            8882              0              100.0
   k562     310385          188590         121795               60.8       8563            8563              0              100.0
   rpe1     247914          173737          74177               70.1       8749            8749              0              100.0


## Examine Gene Lists for Each Dataset


In [5]:
# For each dataset, show gene information
for name, adatas in adata_dict.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {name.upper()}")
    print(f"{'='*60}")
    
    raw = adatas['raw']
    filtered = adatas['filtered']
    
    # Gene names
    raw_genes = set(raw.var_names)
    filtered_genes = set(filtered.var_names)
    
    print(f"\nRaw genes: {len(raw_genes)}")
    print(f"Filtered genes: {len(filtered_genes)}")
    print(f"\nGenes removed: {len(raw_genes - filtered_genes)}")
    print(f"Genes retained: {len(raw_genes & filtered_genes)}")
    
    # Show first 20 genes from each
    print(f"\nFirst 20 raw genes:")
    print(list(raw.var_names[:20]))
    
    print(f"\nFirst 20 filtered genes:")
    print(list(filtered.var_names[:20]))
    
    # Show removed genes (first 20 if any)
    removed_genes = raw_genes - filtered_genes
    if removed_genes:
        print(f"\nFirst 20 removed genes:")
        print(list(removed_genes)[:20])



Dataset: HEPG2

Raw genes: 9624
Filtered genes: 9624

Genes removed: 9624
Genes retained: 0

First 20 raw genes:
['ENSG00000228794', 'ENSG00000188976', 'ENSG00000187583', 'ENSG00000188290', 'ENSG00000187608', 'ENSG00000188157', 'ENSG00000131591', 'ENSG00000078808', 'ENSG00000176022', 'ENSG00000160087', 'ENSG00000131584', 'ENSG00000169972', 'ENSG00000127054', 'ENSG00000224051', 'ENSG00000107404', 'ENSG00000162576', 'ENSG00000175756', 'ENSG00000221978', 'ENSG00000224870', 'ENSG00000242485']

First 20 filtered genes:
['LINC01128', 'NOC2L', 'PLEKHN1', 'HES4', 'ISG15', 'AGRN', 'C1orf159', 'SDF4', 'B3GALT6', 'UBE2J2', 'ACAP3', 'PUSL1', 'INTS11', 'CPTP', 'DVL1', 'MXRA8', 'AURKAIP1', 'CCNL2', 'MRPL20-AS1', 'MRPL20']

First 20 removed genes:
['ENSG00000173473', 'ENSG00000164241', 'ENSG00000159388', 'ENSG00000139546', 'ENSG00000083857', 'ENSG00000171456', 'ENSG00000172831', 'ENSG00000116478', 'ENSG00000115539', 'ENSG00000071082', 'ENSG00000169288', 'ENSG00000127955', 'ENSG00000019995', 'ENSG000

## Check Gene Overlap Across All Datasets


In [6]:
# Get gene sets for all raw and filtered datasets
all_raw_genes = [set(adatas['raw'].var_names) for adatas in adata_dict.values()]
all_filtered_genes = [set(adatas['filtered'].var_names) for adatas in adata_dict.values()]

# Find intersection (common genes across all datasets)
common_raw_genes = set.intersection(*all_raw_genes)
common_filtered_genes = set.intersection(*all_filtered_genes)

print(f"\n=== Gene Overlap Across All Datasets ===")
print(f"Common genes across all RAW datasets: {len(common_raw_genes)}")
print(f"Common genes across all FILTERED datasets: {len(common_filtered_genes)}")
print(f"\nFirst 20 common filtered genes:")
print(sorted(list(common_filtered_genes))[:20])



=== Gene Overlap Across All Datasets ===
Common genes across all RAW datasets: 6642
Common genes across all FILTERED datasets: 6642

First 20 common filtered genes:
['A1BG', 'AAAS', 'AACS', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AAR2', 'AARS', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AATF', 'ABCB10', 'ABCB7', 'ABCB8', 'ABCC1', 'ABCC4', 'ABCC5']


## Detailed View: Select a Specific Dataset


In [7]:
# Choose a dataset to examine in detail
dataset_name = 'hepg2'  # Change this to 'jurkat', 'k562', or 'rpe1'

print(f"\n=== Detailed view of {dataset_name.upper()} ===")
raw = adata_dict[dataset_name]['raw']
filtered = adata_dict[dataset_name]['filtered']

print(f"\nRaw data:")
print(raw)
print(f"\nVar columns: {raw.var.columns.tolist()}")
print(f"\nObs columns: {raw.obs.columns.tolist()}")

print(f"\n{'='*60}")
print(f"\nFiltered data:")
print(filtered)
print(f"\nVar columns: {filtered.var.columns.tolist()}")
print(f"\nObs columns: {filtered.obs.columns.tolist()}")



=== Detailed view of HEPG2 ===

Raw data:
AnnData object with n_obs × n_vars = 145473 × 9624
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'

Var columns: ['gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano']

Obs columns: ['gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line']


Filtered data:
AnnData object with n_obs × n_vars = 96616 × 9624
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'
    uns: 'log1p'

Var columns: ['gene_name', 'chr',

## View Gene Metadata


In [8]:
# Show gene metadata for a specific dataset
dataset_name = 'hepg2'  # Change as needed

print(f"\n=== Gene metadata for {dataset_name.upper()} ===")

print("\nRAW data gene metadata (first 10 genes):")
display(adata_dict[dataset_name]['raw'].var.head(10))

print("\nFILTERED data gene metadata (first 10 genes):")
display(adata_dict[dataset_name]['filtered'].var.head(10))



=== Gene metadata for HEPG2 ===

RAW data gene metadata (first 10 genes):


,gene_name,chr,start,end,class,strand,length,in_matrix,mean,std,cv,fano
gene_id,,,,,,,,,,,,
ENSG00000228794,LINC01128,chr1,825138,868202,gene_version9,+,43064,True,0.187369,0.462334,2.467510,1.140814
ENSG00000188976,NOC2L,chr1,944203,959309,gene_version11,-,15106,True,1.742970,1.967864,1.129029,2.221775
ENSG00000187583,PLEKHN1,chr1,966482,975865,gene_version11,+,9383,True,0.164274,0.444831,2.707855,1.204539
ENSG00000188290,HES4,chr1,998962,1000172,gene_version10,-,1210,True,2.179681,2.977747,1.366139,4.068017
ENSG00000187608,ISG15,chr1,1001138,1014540,gene_version10,+,13402,True,1.701508,4.689206,2.755911,12.923033
ENSG00000188157,AGRN,chr1,1020120,1056118,gene_version15,+,35998,True,0.695437,1.053259,1.514527,1.595190
ENSG00000131591,C1orf159,chr1,1081818,1116361,gene_version17,-,34543,True,0.114778,0.355679,3.098853,1.102198
ENSG00000078808,SDF4,chr1,1216908,1232067,gene_version17,-,15159,True,3.369794,3.116282,0.924769,2.881843
ENSG00000176022,B3GALT6,chr1,1232237,1235041,gene_version7,+,2804,True,0.621670,0.968321,1.557614,1.508271



FILTERED data gene metadata (first 10 genes):


,gene_name,chr,start,end,class,strand,length,in_matrix,mean,std,cv,fano
gene_name_index,,,,,,,,,,,,
LINC01128,LINC01128,chr1,825138,868202,gene_version9,+,43064,True,0.187369,0.462334,2.467510,1.140814
NOC2L,NOC2L,chr1,944203,959309,gene_version11,-,15106,True,1.742970,1.967864,1.129029,2.221775
PLEKHN1,PLEKHN1,chr1,966482,975865,gene_version11,+,9383,True,0.164274,0.444831,2.707855,1.204539
HES4,HES4,chr1,998962,1000172,gene_version10,-,1210,True,2.179681,2.977747,1.366139,4.068017
ISG15,ISG15,chr1,1001138,1014540,gene_version10,+,13402,True,1.701508,4.689206,2.755911,12.923033
AGRN,AGRN,chr1,1020120,1056118,gene_version15,+,35998,True,0.695437,1.053259,1.514527,1.595190
C1orf159,C1orf159,chr1,1081818,1116361,gene_version17,-,34543,True,0.114778,0.355679,3.098853,1.102198
SDF4,SDF4,chr1,1216908,1232067,gene_version17,-,15159,True,3.369794,3.116282,0.924769,2.881843
B3GALT6,B3GALT6,chr1,1232237,1235041,gene_version7,+,2804,True,0.621670,0.968321,1.557614,1.508271


## Export Gene Lists to Files (Optional)


In [ ]:
# Uncomment to export gene lists to text files
# import os
# os.makedirs('gene_lists', exist_ok=True)

# for name, adatas in adata_dict.items():
#     # Export raw genes
#     with open(f'gene_lists/{name}_raw_genes.txt', 'w') as f:
#         f.write('\n'.join(adatas['raw'].var_names))
    
#     # Export filtered genes
#     with open(f'gene_lists/{name}_filtered_genes.txt', 'w') as f:
#         f.write('\n'.join(adatas['filtered'].var_names))

# print("Gene lists exported to gene_lists/ directory")


## Check Specific Gene Presence


In [12]:
# Check if specific genes are present in each dataset
# genes_to_check = ['TP53', 'MYC', 'EGFR', 'BRCA1']  # Change these to genes you're interested in

genes_to_check = ['TBX5', 'TBX20', 'GATA4', 'MEF2A', 'MEF2C', 'MEF2D', 'GATA6', 'MGA', 'TBX3', 'NFIA', 'TGIF2', 'TEAD3', 'MITF', 'TEAD1', 'ESRRB', 'TEAD4', 'NR2F2', 'ESRRA', 'NFIC', 'PBX3']
print("\n=== Gene Presence Check ===")
for gene in genes_to_check:
    print(f"\n{gene}:")
    for name, adatas in adata_dict.items():
        raw_present = gene in adatas['raw'].var_names
        filtered_present = gene in adatas['filtered'].var_names
        print(f"  {name:10s} - Filtered: {filtered_present}")
    
    # Also check processed data if loaded
    try:
        processed_present = gene in processed.var_names
        print(f"  processed  - Present: {processed_present}")
    except NameError:
        pass  # processed not yet loaded



=== Gene Presence Check ===

TBX5:
  hepg2      - Filtered: False
  jurkat     - Filtered: False
  k562       - Filtered: False
  rpe1       - Filtered: False

TBX20:
  hepg2      - Filtered: False
  jurkat     - Filtered: False
  k562       - Filtered: False
  rpe1       - Filtered: False

GATA4:
  hepg2      - Filtered: True
  jurkat     - Filtered: False
  k562       - Filtered: False
  rpe1       - Filtered: False

MEF2A:
  hepg2      - Filtered: True
  jurkat     - Filtered: True
  k562       - Filtered: True
  rpe1       - Filtered: True

MEF2C:
  hepg2      - Filtered: False
  jurkat     - Filtered: False
  k562       - Filtered: True
  rpe1       - Filtered: False

MEF2D:
  hepg2      - Filtered: False
  jurkat     - Filtered: True
  k562       - Filtered: False
  rpe1       - Filtered: True

GATA6:
  hepg2      - Filtered: False
  jurkat     - Filtered: False
  k562       - Filtered: False
  rpe1       - Filtered: True

MGA:
  hepg2      - Filtered: True
  jurkat     - Filter

## Load Processed Data (Combined HVG Dataset)


In [13]:
# Load the processed/combined dataset
print("Loading processed data...")
processed = ad.read_h5ad('processed_data/replogle.h5ad')

print(f"\nProcessed data shape: {processed.shape}")
print(f"Number of cells: {processed.n_obs}")
print(f"Number of genes: {processed.n_vars}")
print(f"\nProcessed data overview:")
print(processed)


Loading processed data...

Processed data shape: (643413, 6642)
Number of cells: 643413
Number of genes: 6642

Processed data overview:
AnnData object with n_obs × n_vars = 643413 × 6642
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_hvg'


## Inspect Processed Data Structure


In [14]:
# Examine the structure of processed data
print("=== Processed Data Structure ===\n")

print("Observation (cell) metadata columns:")
print(processed.obs.columns.tolist())

print("\n\nVariable (gene) metadata columns:")
print(processed.var.columns.tolist())

print("\n\nObsm (observation matrices) keys:")
print(list(processed.obsm.keys()) if processed.obsm.keys() else "None")

print("\n\nVarm (variable matrices) keys:")
print(list(processed.varm.keys()) if processed.varm.keys() else "None")

print("\n\nLayers:")
print(list(processed.layers.keys()) if processed.layers.keys() else "None")


=== Processed Data Structure ===

Observation (cell) metadata columns:
['gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line']


Variable (gene) metadata columns:
['highly_variable', 'means', 'dispersions', 'dispersions_norm']


Obsm (observation matrices) keys:
['X_hvg']


Varm (variable matrices) keys:
None


Layers:
None


## Compare Processed Data Genes with Individual Datasets


In [15]:
# Compare processed data genes with individual filtered datasets
processed_genes = set(processed.var_names)

print("=== Gene Comparison: Processed vs Individual Datasets ===\n")
print(f"Processed data has {len(processed_genes)} genes\n")

for name, adatas in adata_dict.items():
    filtered_genes = set(adatas['filtered'].var_names)
    
    in_both = len(processed_genes & filtered_genes)
    only_in_filtered = len(filtered_genes - processed_genes)
    only_in_processed = len(processed_genes - filtered_genes)
    
    print(f"{name.upper()}:")
    print(f"  Filtered dataset genes: {len(filtered_genes)}")
    print(f"  Genes in both: {in_both}")
    print(f"  Only in filtered: {only_in_filtered}")
    print(f"  Only in processed: {only_in_processed}")
    print()

# Check if processed genes are the common genes across all filtered datasets
print(f"\nProcessed genes == Common filtered genes? {processed_genes == common_filtered_genes}")
print(f"Number of common filtered genes: {len(common_filtered_genes)}")
print(f"Number of processed genes: {len(processed_genes)}")


=== Gene Comparison: Processed vs Individual Datasets ===

Processed data has 6642 genes

HEPG2:
  Filtered dataset genes: 9624
  Genes in both: 6642
  Only in filtered: 2982
  Only in processed: 0

JURKAT:
  Filtered dataset genes: 8882
  Genes in both: 6642
  Only in filtered: 2240
  Only in processed: 0

K562:
  Filtered dataset genes: 8563
  Genes in both: 6642
  Only in filtered: 1921
  Only in processed: 0

RPE1:
  Filtered dataset genes: 8749
  Genes in both: 6642
  Only in filtered: 2107
  Only in processed: 0


Processed genes == Common filtered genes? True
Number of common filtered genes: 6642
Number of processed genes: 6642


## Analyze Highly Variable Genes (HVGs)


In [16]:
# Check HVG information in processed data
print("=== Highly Variable Genes Analysis ===\n")

# Check if HVG column exists
if 'highly_variable' in processed.var.columns:
    n_hvg = processed.var['highly_variable'].sum()
    print(f"Number of highly variable genes: {n_hvg}")
    print(f"Percentage of genes that are HVG: {100 * n_hvg / len(processed.var):.1f}%")
    
    # Get HVG names
    hvg_genes = processed.var_names[processed.var['highly_variable']].tolist()
    print(f"\nFirst 20 HVG genes:")
    print(hvg_genes[:20])
    
    # Show HVG metadata
    print("\n\nHVG metadata (first 10):")
    display(processed.var[processed.var['highly_variable']].head(10))
else:
    print("No 'highly_variable' column found in var metadata")

# Check if X_hvg exists in obsm
if 'X_hvg' in processed.obsm.keys():
    hvg_shape = processed.obsm['X_hvg'].shape
    print(f"\n\nX_hvg matrix shape: {hvg_shape}")
    print(f"  - Number of cells: {hvg_shape[0]}")
    print(f"  - Number of HVG features: {hvg_shape[1]}")
else:
    print("\n\nNo 'X_hvg' found in obsm")


=== Highly Variable Genes Analysis ===

Number of highly variable genes: 2000
Percentage of genes that are HVG: 30.1%

First 20 HVG genes:
['HES4', 'ISG15', 'MIB2', 'CEP104', 'RERE', 'ENO1', 'SLC25A33', 'CTNNBIP1', 'KIF1B', 'KIAA2013', 'PRDM2', 'SPEN', 'NBPF1', 'RCC2', 'CAMK2N1', 'DDOST', 'RPL11', 'LYPLA2', 'GALE', 'HMGCL']


HVG metadata (first 10):


,highly_variable,means,dispersions,dispersions_norm
gene_name_index,,,,
HES4,True,0.693763,1.156615,3.630171
ISG15,True,0.756257,1.104892,3.423880
MIB2,True,0.256947,0.251591,2.033099
CEP104,True,0.322283,0.117742,0.588443
RERE,True,0.392563,0.194411,0.213229
ENO1,True,2.744879,1.348071,0.287540
SLC25A33,True,0.567329,0.507235,2.071305
CTNNBIP1,True,0.452744,0.450419,1.733833
KIF1B,True,0.393156,0.198320,0.236451




X_hvg matrix shape: (643413, 2000)
  - Number of cells: 643413
  - Number of HVG features: 2000


In [23]:
genes_to_check = ['TBX5', 'TBX20', 'GATA4', 'MEF2A', 'MEF2C', 'MEF2D', 'GATA6', 'MGA', 'TBX3', 'NFIA', 'TGIF2', 'TEAD3', 'MITF', 'TEAD1', 'ESRRB', 'TEAD4', 'NR2F2', 'ESRRA', 'NFIC', 'PBX3']
print("\n=== Gene Presence Check ===")
count = 0
for gene in genes_to_check:
    print(f"\n{gene}:")
    if gene in hvg_genes:
        count += 1
        print("True!")
    else:
        print("False")
print(f"Number of genes overlapped: {count}")
# hvg_genes


=== Gene Presence Check ===

TBX5:
False

TBX20:
False

GATA4:
False

MEF2A:
True!

MEF2C:
False

MEF2D:
False

GATA6:
False

MGA:
False

TBX3:
False

NFIA:
True!

TGIF2:
False

TEAD3:
False

MITF:
False

TEAD1:
True!

ESRRB:
False

TEAD4:
False

NR2F2:
False

ESRRA:
False

NFIC:
True!

PBX3:
False
Number of genes overlapped: 4


## View Processed Data Genes


In [17]:
# Show genes in processed data
print("=== Processed Data Gene List ===\n")

print(f"Total genes: {len(processed.var_names)}")
print(f"\nFirst 50 genes:")
print(list(processed.var_names[:50]))

print(f"\n\nLast 20 genes:")
print(list(processed.var_names[-20:]))

# Show gene metadata
print("\n\nGene metadata (first 10 genes):")
display(processed.var.head(10))

print("\n\nGene metadata (last 10 genes):")
display(processed.var.tail(10))


=== Processed Data Gene List ===

Total genes: 6642

First 50 genes:
['NOC2L', 'HES4', 'ISG15', 'SDF4', 'B3GALT6', 'UBE2J2', 'ACAP3', 'PUSL1', 'INTS11', 'DVL1', 'AURKAIP1', 'CCNL2', 'MRPL20-AS1', 'MRPL20', 'ATAD3B', 'ATAD3A', 'SSU72', 'MIB2', 'CDK11B', 'CDK11A', 'NADK', 'GNB1', 'FAAP20', 'SKI', 'RER1', 'WRAP73', 'LRRC47', 'CEP104', 'C1orf174', 'RPL22', 'ICMT', 'ACOT7', 'NOL9', 'KLHL21', 'PHF13', 'THAP3', 'DNAJC11', 'CAMTA1', 'VAMP3', 'PARK7', 'RERE', 'ENO1', 'SLC25A33', 'CLSTN1', 'CTNNBIP1', 'LZIC', 'UBE4B', 'KIF1B', 'PGD', 'CENPS']


Last 20 genes:
['DKC1', 'MPP1', 'FUNDC2', 'CMC4', 'BRCC3', 'VBP1', 'VAMP7', 'MT-ND1', 'MT-ND2', 'MT-CO1', 'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5', 'MT-ND6', 'MT-CYB']


Gene metadata (first 10 genes):


,highly_variable,means,dispersions,dispersions_norm
gene_name_index,,,,
NOC2L,False,0.690140,0.089601,-0.625512
HES4,True,0.693763,1.156615,3.630171
ISG15,True,0.756257,1.104892,3.423880
SDF4,False,0.706835,0.167762,-0.313772
B3GALT6,False,0.283483,0.041215,-0.237526
UBE2J2,False,0.542493,0.016646,-0.842638
ACAP3,False,0.150128,0.028836,-0.371131
PUSL1,False,0.229711,-0.057846,-1.306713
INTS11,False,0.375829,-0.028033,-1.108016




Gene metadata (last 10 genes):


,highly_variable,means,dispersions,dispersions_norm
gene_name_index,,,,
MT-CO2,False,4.899232,3.297817,-0.707107
MT-ATP8,True,0.486560,0.276719,0.702114
MT-ATP6,True,4.781817,3.148722,0.707107
MT-CO3,True,5.040968,3.838709,0.707107
MT-ND3,True,3.656196,2.475402,2.997896
MT-ND4L,True,1.379477,0.915507,0.915423
MT-ND4,True,4.453991,2.885937,0.192740
MT-ND5,True,2.873720,1.403472,0.412011
MT-ND6,True,1.812226,1.427944,1.723333


## Analyze Cell Composition in Processed Data


In [18]:
# Analyze cells in processed data
print("=== Cell Composition in Processed Data ===\n")

print(f"Total cells: {processed.n_obs}")

# Show first few rows of cell metadata
print("\n\nCell metadata (first 10 cells):")
display(processed.obs.head(10))

# If there's a batch or dataset identifier, show composition
# Common column names for batch/dataset info
possible_batch_cols = ['batch', 'dataset', 'sample', 'cell_type', 'celltype', 'condition']

for col in possible_batch_cols:
    if col in processed.obs.columns:
        print(f"\n\nCell counts by '{col}':")
        counts = processed.obs[col].value_counts()
        print(counts)
        print(f"\nPercentages:")
        print(processed.obs[col].value_counts(normalize=True) * 100)


=== Cell Composition in Processed Data ===

Total cells: 643413


Cell metadata (first 10 cells):


,gem_group,gene,gene_id,transcript,gene_transcript,sgID_AB,mitopercent,UMI_count,z_gemgroup_UMI,cell_line
cell_barcode,,,,,,,,,,
AAACCCAAGAATAGTC-3,3,KIAA1143,ENSG00000163807,P1P2,4360_KIAA1143_P1P2_ENSG00000163807,KIAA1143_+_44803075.23-P1P2|KIAA1143_+_4480308...,0.114029,11234.0,-0.611091,hepg2
AAACCCAAGAGGTATT-55,55,SEPHS2,ENSG00000179918,P1P2,7779_SEPHS2_P1P2_ENSG00000179918,SEPHS2_-_30457178.23-P1P2|SEPHS2_+_30457164.23...,0.165242,45146.0,0.208833,hepg2
AAACCCAAGAGTGACC-39,39,POLR2H,ENSG00000163882,P1P2,6549_POLR2H_P1P2_ENSG00000163882,POLR2H_+_184081227.23-P1P2|POLR2H_-_184081237....,0.162209,20190.0,-0.025114,hepg2
AAACCCAAGATGGCAC-43,43,TFAM,ENSG00000108064,P1P2,8832_TFAM_P1P2_ENSG00000108064,TFAM_+_60145205.23-P1P2|TFAM_-_60145223.23-P1P2,0.071596,23912.0,2.079665,hepg2
AAACCCAAGCAACAAT-16,16,TMEM214,ENSG00000119777,P1P2,9042_TMEM214_P1P2_ENSG00000119777,TMEM214_-_27255873.23-P1P2|TMEM214_-_27255853....,0.109515,8282.0,-0.214576,hepg2
AAACCCAAGCACACAG-14,14,BAP1,ENSG00000163930,P1P2,760_BAP1_P1P2_ENSG00000163930,BAP1_+_52443980.23-P1P2|BAP1_+_52443883.23-P1P2,0.150652,33209.0,0.083693,hepg2
AAACCCAAGCGACTGA-47,47,MAP2K7,ENSG00000076984,P1P2,4805_MAP2K7_P1P2_ENSG00000076984,MAP2K7_-_7968835.23-P1P2|MAP2K7_+_7968839.23-P1P2,0.097892,15895.0,-0.089397,hepg2
AAACCCAAGCGCCATC-14,14,UXT,ENSG00000126756,P1P2,9691_UXT_P1P2_ENSG00000126756,UXT_+_47518485.23-P1P2|UXT_-_47518537.23-P1P2,0.105686,24743.0,-0.496336,hepg2
AAACCCAAGCTCCACG-12,12,SHQ1,ENSG00000144736,P1P2,7903_SHQ1_P1P2_ENSG00000144736,SHQ1_+_72897533.23-P1P2|SHQ1_+_72897517.23-P1P2,0.095725,13288.0,0.325491,hepg2


## Summary: All Datasets Overview


In [ ]:
# Create comprehensive summary including processed data
summary_data = []

# Add individual datasets
for name, adatas in adata_dict.items():
    summary_data.append({
        'Dataset': f"{name} (raw)",
        'Cells': adatas['raw'].n_obs,
        'Genes': adatas['raw'].n_vars,
        'Type': 'Raw'
    })
    summary_data.append({
        'Dataset': f"{name} (filtered)",
        'Cells': adatas['filtered'].n_obs,
        'Genes': adatas['filtered'].n_vars,
        'Type': 'Filtered'
    })

# Add processed data
summary_data.append({
    'Dataset': 'processed (combined)',
    'Cells': processed.n_obs,
    'Genes': processed.n_vars,
    'Type': 'Processed'
})

summary_df = pd.DataFrame(summary_data)

print("=== Complete Dataset Summary ===\n")
print(summary_df.to_string(index=False))

# Calculate totals
print("\n\n=== Totals ===")
print(f"Total raw cells: {summary_df[summary_df['Type'] == 'Raw']['Cells'].sum()}")
print(f"Total filtered cells: {summary_df[summary_df['Type'] == 'Filtered']['Cells'].sum()}")
print(f"Processed cells: {processed.n_obs}")
print(f"\nCommon genes across all filtered datasets: {len(common_filtered_genes)}")
print(f"Genes in processed data: {processed.n_vars}")
